In [1]:
#imports
import astropy.units as u
import numpy as np
import gala.potential as gp
import gala.dynamics as gd
from gala.dynamics import mockstream as ms
from gala.units import galactic
from astropy.coordinates import SkyCoord, Distance
#import popsims
import matplotlib.pyplot as plt
#%matplotlib notebook
from tqdm import tqdm
from astropy.constants import G
import HSS
from gala.units import galactic
import gala.coordinates as gc
import  h5py
import imageio
import matplotlib as mpl
mpl.rc('xtick', labelsize=16) 
mpl.rc('ytick', labelsize=16) 
font = {#'family' : 'Helvetica',
        'size'   : 16}

In [2]:
import astropy.coordinates as coord
_ = coord.galactocentric_frame_defaults.set('v4.0')

pot=gp.MilkyWayPotential()
H = gp.Hamiltonian(pot)

In [6]:
#initialize satetllite position
st_pos = gd.PhaseSpacePosition(pos=[10., 0., 0.] * u.kpc, vel=[0, 170, 0.] * u.km/u.s)
mstream=5e5
tmin=0.0
tmax=500.
nsteps=1000
time_dict={'t': np.linspace(tmin, tmax, nsteps)*u.Myr}

In [7]:
#call mockstream generator, integrate stream forard in time
df = ms.FardalStreamDF()
prog_mass = mstream* u.Msun
gen = ms.MockStreamGenerator(df, H)
mock_st, mock_pos=gen.run(st_pos, prog_mass, **time_dict, nbody=None, progress=True)

Integrating orbits:  100%

In [10]:
#this is the place for collision
collision_pos=mock_st.cartesian.xyz[:,-150]
#give it a velocity
collision_phase_space_pos=gd.PhaseSpacePosition(pos=collision_pos,
                                vel=[0, 0, -500] * u.km/u.s)
#integrate it back
collision_orbit= gp.Hamiltonian(pot).integrate_orbit(collision_phase_space_pos,  dt=-1*u.Myr,\
                                                     t1=0.*u.Myr, t2=-tmax*u.Myr)

In [16]:
collision_pos, collision_orbit.xyz[:, 0],  collision_orbit.xyz[:, -1]

(<Quantity [-2.88036484,  7.74214996, -0.03328881] kpc>,
 <Quantity [-2.88036484,  7.74214996, -0.03328881] kpc>,
 <Quantity [ 14.75134622, -39.65023213, 119.48625245] kpc>)

In [19]:
collision_orbit[-1].v_xyz #it's not moving at -500 km/s but at 

<Quantity [-0.02918847,  0.07845585, -0.1394293 ] kpc / Myr>

In [12]:
#integrate this in time just to check
gp.Hamiltonian(pot).integrate_orbit(collision_phase_space_pos,  dt=-1*u.Myr,\
                                                     t1=0.*u.Myr, t2=-tmax*u.Myr)

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,